#### Defining function to get fact data

In [0]:
@keep_log
def get_hourly_fact_weather(df):
    
    from datetime import datetime
    from pyspark.sql.functions import hour, col
    
    df = df.withColumn('timeID', hour('created_on')).withColumn('Date', col('created_on').cast('date'))
    
    try:
        # df_hourly_fact = spark.table('hourly_fact_weather')
        timeID, date = df.select('timeID', 'Date').first()
        dateID = str(date).replace('-', '')
        query = f"delete from hourly_fact_weather where timeID='{timeID}' and dateID='{dateID}';"
        spark.sql(query)
    except:
        pass
    
    df_date = spark.table('dim_date')
    df_fact = df.join(df_date, df.Date == df_date.fullDate).select(
                            df.timeID,
                            df_date.dateID,
                            df.city_id,
                            df.temp,
                            df.temp_min,
                            df.temp_max,
                            df.pressure,
                            df.humidity,
                            df.visibility,
                            df.wind_speed,
                            df.wind_deg,
                            df.wind_gust,
                            df.clouds_all
                        )
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    
    return df_fact, start, end

In [0]:
# df_log = spark.table('weather_log_table')
# df_log = df_log.where(col("load_type")=='PROCESSED').sort('created_on', ascending=False)

# df = spark.table('processed_weather')
# a, b = df.withColumn('timeID', hour('created_on')).withColumn('Date', col('created_on').cast('date')).sort('timeID', ascending=False).limit(5).select('timeID', 'Date').first()
# display(df_log)
# display(df)
# # display(df_new)
# b= str(b).replace('-', '') 
# print(a, b)


# # query = f"select * from (select *, to_date(date) as nd from processed_weather) where nd='{b}'"
# # display(spark.sql(query))